# Task2.c - Update expenditure figures from commission payments as per the previous script

We here apply the script implemented in the previous tender contract to estimate expenditures also for the programming period 2014-2020.

We start off from importing the relevant libraries

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pkg_resources
import pandas as pd
import numpy as np
import sobol_seq
import types

And the relevant datasets.

In [2]:
df = pd.read_excel('Database_Final_UPD_2020_corr.xlsx')
df_1420 = pd.read_excel('2019-07-18_2014-2020_regionalised.xlsx')
df_1420['ProgrammingPeriod']='2014-2020'
df_1420 = df_1420.fillna(method='ffill')
df_1420 = df_1420.rename(columns={'MS':'Country','NUTS1':'NUTS1Code','NUTS2':'NUTS2Code','CF':'CF_TOTAL','ERDF':'ERDF_TOTAL'})
df_old = pd.read_csv('Old_data.csv')

Quasi-random numbers are then generated for our Monte Carlo simulations. The expenditure algorithm is not run only once, rather multiple times to obtain plausible estimates of the expenditure ranges.

In [3]:
quasiRandom_df = pd.DataFrame(sobol_seq.i4_sobol_generate(6,1000))
DistributionFiMax = 0.8+quasiRandom_df[0]*0.2
DistributionFiMin = 0.2+quasiRandom_df[1]*0.2

Dataframes are re-arranged for our convenience

In [4]:
dfb = df[['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year','CF_TOTAL','EAGGF','ERDF_TOTAL','ESF']].copy()
df2 = dfb.melt(id_vars=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year'],var_name='FundingScheme')
df3 = df2.pivot_table(index=['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code'],columns='Year', values='value')
df4 = df3.dropna(how='all').fillna(0)

df_14b = df_1420.melt(id_vars=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year'],var_name='FundingScheme')
df_14c = df_14b.pivot_table(index=['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code'],columns='Year',
                            values='value')
df_14d = pd.concat([df3,df_14c])
df4 = df_14d.dropna(how='all').fillna(0)

Anticipated payments, i.e. remitted before the commencing of the programming period, are excluded from the assessment. For instance, a payment remitted in the year 2006 in the context of the programming period 2007-2013 is attributed to an expenditure incurred in the same year.

In [5]:
dummy = []

diff = int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][5:])+6-\
int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][:4])

d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in \
range(int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][:4]),
int(np.sort(np.array(list(set(df.ProgrammingPeriod))))[0][5:])+6)])

dummy.append(d_Var)

for pp in np.sort(np.array(list(set(df.ProgrammingPeriod))))[1:-1]:
    diff = int(pp[5:])+4-int(pp[:4])
    
    d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in range(int(pp[:4]),int(pp[5:])+4)])
    
    dummy.append(d_Var)
    
for pp in np.sort(np.array(list(set(df.ProgrammingPeriod))))[-1:]:
    diff = int(pp[5:])+3-int(pp[:4])
    
    d_Var = pd.Series([k/diff for k in range(1,diff+1)],[y for y in range(int(pp[:4]),int(pp[5:])+3)])
    
    dummy.append(d_Var)
    
dummy.append(pd.Series([r/(2019-2014) for r in range(1,1+2019-2014)], [y for y in range(2014,2019)], dtype='float64'))

Cumulative figures are then normalised.

In [6]:
df5 = df4.copy()

Norm_df6 = ((df5.cumsum(axis=1).T/df5.cumsum(axis=1).max(axis=1).values).T).dropna(how='all')

In [7]:
Norm_df6.loc[pd.IndexSlice['2014-2020',:,:,:],'$\delta$']= (Norm_df6.loc['2014-2020',dummy[-1].index]-\
                                                            dummy[-1].T).cumsum(axis=1).iloc[:,-1]

Two outcome variables are the defined for our assessment: a) $\delta$ which represents the difference in the cumulative areas of the payments against a dummy region; and b) $\mu$ which is a normalised rank of the European regions for a given funding scheme under a given programming period.

In [8]:
lp = list(set(df.ProgrammingPeriod))
lp.append('2014-2020')

for ip, pp in enumerate(np.sort(np.array(lp))):
    Norm_df6.loc[pd.IndexSlice[pp,:,:,:],'$\delta$'] = (Norm_df6.loc[pd.IndexSlice[pp,:,:,:],dummy[ip].index]-\
                                                        dummy[ip].T).cumsum(axis=1).iloc[:,-1]    

In [9]:
df7 = Norm_df6.groupby(['ProgrammingPeriod','FundingScheme'])
ld = []
for idx, df7b in df7:
    df8 = df7b.copy()
    df8['$\mu$']=(df8['$\delta$'].max()-df8['$\delta$'])/(df8['$\delta$'].max()-df8['$\delta$'].min())
    ld.append(df8)
Norm_df7 = pd.concat(ld)

A further modelling parameter is then defined: the trigger for the number of years the residual expenditure gets spread onto on the last eligible year of the programming period.

In [10]:
df7c = Norm_df7.groupby(['ProgrammingPeriod'])
ld1 = []
co = -1
for idx, df7d in df7c:
    co+=1
    for iq,qr in enumerate(quasiRandom_df[2]):
        df8b = df7d.copy()
        df8b[0]=(qr*(df7d['$\mu$']*(len(dummy[co])-1)).astype(int)).astype(int)+1
        for il in range(1,len(dummy[co])):
            df8b[il]=df8b[0]-il
        df8b[df8b<1]=1
        cd = [il0 for il0 in range(len(dummy[co]))]
        df8b['value']=iq
        cd.append('value')
        df8b = df8b[cd]
        ld1.append(df8b)
years = pd.concat(ld1)
years.set_index('value', append=True, inplace=True)

Here we define the yearly residues a payment may get attributed to. After having defined which share of the payment correspond to an expenditure incurred on the same year, we attribute the residual backwards to expenditure incurred in the previous years as per the figures assessed below. 

In [11]:
def A(n):
    return [(2**j)/(2**n-1) for j in reversed(range(n))]

B9 = []
for k in reversed(range(2,11)):
    B9.append(pd.DataFrame([A(y) for y in range(1,k)],index=[y for y in range(1,k)],
                           columns=[y for y in range(1,k)]).fillna(0).sort_values(by=1,ascending=False))

In [12]:
Payments = df5.loc[Norm_df6.index].copy()

The expenditure figures can then be assessed in a Monte Carlo simulation (repeated 1,000 times).

In [13]:
eg  = df5.loc[Norm_df6.index].groupby('ProgrammingPeriod')
co = -1

Expend = []
for r in range(len(quasiRandom_df)):
    co = -1
    Expe = []
    for ie, Eg in eg:
        co += 1
        Exp = Eg.copy()
        if ie == '1989-1993':
            y = [yr for yr in range(int(ie[5:])+5,df5.columns.max())]

            Exp[int(ie[5:])+5]=Eg[y].agg('sum',axis=1)
            
            Exp.loc[:,y[1:]] = 0
            
            Aggregate = Exp[int(ie[5:])+5]*(1-(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r])))
            
            Exp[int(ie[5:])+5] = Exp[int(ie[5:])+5]*(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))
            for iy2,y2 in enumerate(range(int(ie[:4]),int(ie[5:])+5)):
                Exp[y2] = Eg[y2]*(DistributionFiMax[r]- 
                Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))
                
                for iy3,y3 in enumerate(reversed(range(y2,int(ie[5:])+4))):
                    Exp[y2]+=Eg[y3+1]*(1-DistributionFiMax[r]+ 
                    Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))*\
                    B9[iy3+len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],iy3].values,
                                                     (y3+1-y2)].values
                    
                Exp[y2] += Aggregate*B9[len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],0].values,
                                                                         len(B9[len(dummy[0])-len(dummy[co])])-iy2].values
                
            Exp[int(ie[:4])] += Eg[int(ie[:4])]*(1-DistributionFiMax[r]+
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

            
        elif ie == '1994-1999' or ie == '2000-2006':
            y = [yr for yr in range(int(ie[5:])+3,df5.columns.max())]

            Exp[int(ie[5:])+3]=Eg[y].agg('sum',axis=1)

            Exp.loc[:,y[1:]] = 0
            
            Aggregate = Exp[int(ie[5:])+3]*(1-(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r])))

            Exp[int(ie[5:])+3] = Exp[int(ie[5:])+3]*(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

            for iy2,y2 in enumerate(range(int(ie[:4]),int(ie[5:])+3)):
                Exp[y2] = Eg[y2]*(DistributionFiMax[r]- 
                Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))
                
                for iy3,y3 in enumerate(reversed(range(y2,int(ie[5:])+2))):
                    Exp[y2]+=Eg[y3+1]*(1-DistributionFiMax[r]+ 
                    Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))*\
                    B9[iy3+len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],iy3].values,
                                                     (y3+1-y2)].values
                Exp[y2] += Aggregate*B9[len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],0].values,
                                                                         len(B9[len(dummy[0])-len(dummy[co])])-iy2].values
            
            Exp[int(ie[:4])] += Eg[int(ie[:4])]*(1-DistributionFiMax[r]+
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))
         
        elif ie == '2007-2013':
            y = [yr for yr in range(int(ie[5:])+2,df5.columns.max()+1)]

            Exp[int(ie[5:])+2]=Eg[y].agg('sum',axis=1)
            Exp.loc[:,y[1:]] = 0
            
            Aggregate = Exp[int(ie[5:])+2]*(1-(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r])))

            Exp[int(ie[5:])+2] = Exp[int(ie[5:])+2]*(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

            for iy2,y2 in enumerate(range(int(ie[:4]),int(ie[5:])+2)):
                Exp[y2] = Eg[y2]*(DistributionFiMax[r]- 
                Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))
                
                for iy3,y3 in enumerate(reversed(range(y2,int(ie[5:])+1))):
                    Exp[y2]+=Eg[y3+1]*(1-DistributionFiMax[r]+ 
                    Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))*\
                    B9[iy3+len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],iy3].values,
                                                     (y3+1-y2)].values
                Exp[y2] += Aggregate*B9[len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],0].values,
                                                                         len(B9[len(dummy[0])-len(dummy[co])])-iy2].values
            
            Exp[int(ie[:4])] += Eg[int(ie[:4])]*(1-DistributionFiMax[r]+
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))
        
        else:
            Exp[df5.columns.max()]=Eg[df5.columns.max()]*(DistributionFiMax[r]- 
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

            for iy2,y2 in enumerate(reversed(range(int(ie[:4]),df5.columns.max()))):
                Exp[y2] = Eg[y2]*(DistributionFiMax[r]- 
                Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))

                for iy3,y3 in enumerate(reversed(range(y2,df5.columns.max()))):
                    Exp[y2]+=Eg[y3+1]*(1-DistributionFiMax[r]+ 
                    Norm_df7.loc[pd.IndexSlice[ie,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))*\
                    B9[iy3+len(dummy[0])-len(dummy[co])].loc[years.loc[pd.IndexSlice[ie,:,:,:,:,r],iy3].values,
                                                     (y3+1-y2)].values

            Exp[int(ie[:4])] += Eg[int(ie[:4])]*(1-DistributionFiMax[r]+
            Norm_df7.loc[pd.IndexSlice[ie,:,:,:,:],'$\mu$']*(DistributionFiMax[r]-DistributionFiMin[r]))
        
        Expe.append(Exp)
    Expen = pd.concat(Expe)
    Expen['r']=r
    Expend.append(Expen)
Expenditure = pd.concat(Expend)
Expenditure.set_index('r', append=True, inplace=True)

The database originated can be finally exported.

In [14]:
Expenditure.to_csv('Expenditure_including_2014-2020.csv')

The data is eventually harmonised and merged into a single database.

In [15]:
Mean = Expenditure.groupby(['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code']).mean()
Std = Expenditure.groupby(['ProgrammingPeriod','FundingScheme','Country','NUTS1Code','NUTS2Code']).std()

In [16]:
Mean_unstuck = pd.pivot_table(Mean.stack().reset_index(),values = 0,columns = 'FundingScheme',
                                        index=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year']).reset_index()
Std_unstuck = pd.pivot_table(Std.stack().reset_index(),values = 0,columns = 'FundingScheme',
                                        index=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year']).reset_index()
Payment_unstuck = pd.pivot_table(df4.stack().reset_index(),values = 0,columns = 'FundingScheme',
                                        index=['ProgrammingPeriod','Country','NUTS1Code','NUTS2Code','Year']).reset_index()

In [17]:
Mean_unstuck = Mean_unstuck.rename(columns={'ProgrammingPeriod':'Programming_Period','NUTS1Code':'NUTS1_ID','NUTS2Code':'NUTS2_ID',
                        'ERDF_TOTAL':'ERDF','CF_TOTAL':'CF'})
Std_unstuck = Std_unstuck.rename(columns={'ProgrammingPeriod':'Programming_Period','NUTS1Code':'NUTS1_ID','NUTS2Code':'NUTS2_ID',
                        'ERDF_TOTAL':'ERDF','CF_TOTAL':'CF'})
Payment_unstuck = Payment_unstuck.rename(columns={'ProgrammingPeriod':'Programming_Period','NUTS1Code':'NUTS1_ID','NUTS2Code':'NUTS2_ID',
                        'ERDF_TOTAL':'ERDF','CF_TOTAL':'CF'})

In [18]:
Mean_melt = pd.melt(Mean_unstuck, id_vars=['Programming_Period','Country','NUTS1_ID','NUTS2_ID','Year'], value_vars=['ERDF', 'CF',
        'EAGGF','ESF','CF','EAFRD','EMFF','FEAD','YEI'],var_name='Fund', value_name='Modelled_annual_expenditure')
Std_melt = pd.melt(Std_unstuck, id_vars=['Programming_Period','Country','NUTS1_ID','NUTS2_ID','Year'], value_vars=['ERDF', 'CF',
        'EAGGF','ESF','CF','EAFRD','EMFF','FEAD','YEI'],var_name='Fund', value_name='Standard_deviation_of_annual_expenditure')
Payments_melt = pd.melt(Payment_unstuck, id_vars=['Programming_Period','Country','NUTS1_ID','NUTS2_ID','Year'], 
        value_vars=['ERDF', 'CF','EAGGF','ESF','CF','EAFRD','EMFF','FEAD','YEI'],var_name='Fund', value_name='EU_Payment_annual')

In [19]:
Exp_df = pd.concat([Mean_melt,Std_melt.Standard_deviation_of_annual_expenditure],axis=1)

In [20]:
Output = Exp_df.join(Payments_melt.set_index(['Programming_Period','Country','NUTS1_ID','NUTS2_ID','Year','Fund']),
            on=['Programming_Period','Country','NUTS1_ID','NUTS2_ID','Year','Fund']).fillna(0)

NUTS nomenclature is finally harmonised

In [21]:
Output.iloc[:,-3:]=Output.iloc[:,-3:].astype(int)

mapping_cat = dict(df_old[['NUTS2_ID', 'NUTS2_name']].values)
mapping_cat['EL11']='Anatoliki Makedonia,Thraki'
mapping_cat['EL12']='Kentriki Makedonia'
mapping_cat['EL13']='Dytiki Makedonia'
mapping_cat['EL14']='Thessalia'
mapping_cat['EL21']='Ipeiros'
mapping_cat['EL22']='Ionia Nisia'
mapping_cat['EL23']='Dytiki Ellada'
mapping_cat['EL24']='Sterea Ellada'
mapping_cat['EL25']='Peloponnisos'
mapping_cat['FRZZ']='Extra-Regio NUTS 2'
mapping_cat['SI01']='Vzhodna Slovenija'
mapping_cat['SI02']='Zahodna Slovenija'
mapping_cat['UKZZ']='Extra-Regio NUTS 2'
Output['NUTS2_name'] = Output['NUTS2_ID'].map(mapping_cat)

The formatting is set as per the instruction received.

In [22]:
Output = Output[['Country','NUTS1_ID','NUTS2_ID','NUTS2_name','Fund','Year','Programming_Period','EU_Payment_annual',
                  'Modelled_annual_expenditure','Standard_deviation_of_annual_expenditure']].drop_duplicates()

In [23]:
Output

,Country,NUTS1_ID,NUTS2_ID,NUTS2_name,Fund,Year,Programming_Period,EU_Payment_annual,Modelled_annual_expenditure,Standard_deviation_of_annual_expenditure
0,BE,BE1,BE10,Région de Bruxelles-Capitale / Brussels Hoofds...,ERDF,1986,1989-1993,0,0,0
1,BE,BE1,BE10,Région de Bruxelles-Capitale / Brussels Hoofds...,ERDF,1987,1989-1993,0,0,0
2,BE,BE1,BE10,Région de Bruxelles-Capitale / Brussels Hoofds...,ERDF,1988,1989-1993,0,0,0
3,BE,BE1,BE10,Région de Bruxelles-Capitale / Brussels Hoofds...,ERDF,1989,1989-1993,0,0,0
4,BE,BE1,BE10,Région de Bruxelles-Capitale / Brussels Hoofds...,ERDF,1990,1989-1993,0,3253,4051
...,...,...,...,...,...,...,...,...,...,...
362929,UK,UKZ,UKZZ,Extra-Regio NUTS 2,YEI,2014,2014-2020,0,0,0
362930,UK,UKZ,UKZZ,Extra-Regio NUTS 2,YEI,2015,2014-2020,0,0,0
362931,UK,UKZ,UKZZ,Extra-Regio NUTS 2,YEI,2016,2014-2020,0,0,0
362932,UK,UKZ,UKZZ,Extra-Regio NUTS 2,YEI,2017,2014-2020,0,0,0


And the relevant statistical properties of the distributions calculated (mean and standard deviation) can be also calculated and exported in a datasheet.

In [24]:
Output.to_excel('PivotedData_including_2014-2020.xlsx')